In [ ]:
import numpy as np
import json
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation

from lac.localization.imu_recovery import recover_rotation_exact, recover_translation

%load_ext autoreload
%autoreload 2

### Load the data


In [122]:
data_path = "../../output/nav_agent/data_log.json"
json_data = json.load(open(f"{data_path}"))

poses = []
imu_data = []

for frame in json_data["frames"]:
    poses.append(np.array(frame["pose"]))
    imu_data.append(np.array(frame["imu"]))

imu_data = np.array(imu_data)
initial_pose = np.array(json_data["initial_pose"])

gt_translations = np.zeros((len(poses), 3))
for i in range(len(poses)):
    gt_translations[i] = poses[i][:3, 3]

dt = 0.05
times = np.arange(0, len(imu_data) * dt, dt)

### Initialization


In [123]:
start_idx = 0
poses_crop = poses[start_idx:]

In [124]:
# R_prev = poses[start_idx][:3, :3]
# # t_prev_prev = poses[start_idx-1][:3, 3]
# t_prev_prev = poses[start_idx][:3, 3]
# t_prev = poses[start_idx][:3, 3]

R_prev = initial_pose[:3, :3]
t_prev_prev = initial_pose[:3, 3]
t_prev = initial_pose[:3, 3]

In [ ]:
initial_pose - poses[0]

In [ ]:
imu_rotations = []
imu_rotations.append(R_prev)

imu_translations = []
imu_translations.append(t_prev)

for i in tqdm(range(start_idx + 1, len(poses))):
    # Rotation recovery
    omega = imu_data[i, 3:]
    R_curr = recover_rotation_exact(R_prev, omega, dt)
    imu_rotations.append(R_curr)
    R_prev = R_curr

    # Translation recovery
    a = imu_data[i, :3]
    t_curr = recover_translation(t_prev_prev, t_prev, R_curr, a, dt)
    imu_translations.append(t_curr)
    t_prev_prev = t_prev
    t_prev = t_curr

In [ ]:
gt_rotations_euler = np.zeros((len(poses_crop), 3))
imu_rotations_euler = np.zeros((len(poses_crop), 3))

for i in range(len(poses_crop)):
    gt_rotations_euler[i] = Rotation.from_matrix(poses_crop[i][:3, :3]).as_euler(
        "xyz", degrees=True
    )
    imu_rotations_euler[i] = Rotation.from_matrix(imu_rotations[i]).as_euler("xyz", degrees=True)

fig, axes = plt.subplots(3, 1, figsize=(12, 8))
pos_labels = ["Roll (deg)", "Pitch (deg)", "Yaw (deg)"]
for i in range(3):
    ax = axes[i]
    ax.plot(times, gt_rotations_euler[:, i], label="True")
    ax.plot(times, imu_rotations_euler[:, i], label="IMU", alpha=0.5)
    ax.legend()
    ax.grid()
    ax.set_xlabel("Time (s)")
    ax.set_ylabel(f"{pos_labels[i]}")
plt.subplots_adjust(wspace=0.0, hspace=0.3)

print("Average error in degrees")
print(np.mean(np.abs(gt_rotations_euler - imu_rotations_euler), axis=0))

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 8))
pos_labels = ["X", "Y", "Z"]

imu_translations = np.array(imu_translations)

gt_translations = np.zeros((len(poses_crop), 3))
for i in range(len(poses_crop)):
    gt_translations[i] = poses_crop[i][:3, 3]

for i in range(3):
    ax = axes[i]
    ax.plot(times, gt_translations[:, i], label="True")
    ax.plot(times, imu_translations[:, i], label="IMU", alpha=0.5)
    ax.legend()
    ax.grid()
    ax.set_xlabel("Time (s)")
    ax.set_ylabel(f"{pos_labels[i]}")
plt.subplots_adjust(wspace=0.0, hspace=0.3)

print("Average error in meters")
print(np.mean(np.abs(gt_translations - imu_translations), axis=0))

Average error in meters
[0.0023551 0.00487077 0.00072259]
